# V17: Ephemeris reliability (null tests + neighborhood)

Tests whether the signal is uniquely concentrated at the expected phase and whether nearby periods produce comparable scores.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V17')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': list(r.flags),
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "depth_hat_ppm": 252.57172720708812,
    "depth_sigma_ppm": 6.72726577230079,
    "effective_n_points": 770.8631381129171,
    "max_ablation_score_drop_fraction": 0.018014316033769004,
    "n_in_transit": 972,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood_best_period_days": 14.2423724,
    "period_neighborhood_best_score": 37.54448475144845,
    "period_neighborhood_second_best_score": 31.101255013267536,
    "period_peak_to_next_ratio": 1.2071694449446586,
    "phase_shift_null_p_value": 0.004975124378109453,
    "phase_shift_null_z": 6.1268661178199455,
    "score": 37.54448475144845,
    "top_5_fraction_abs": 0.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_ephemeris_reliability

step_id = '19_v17_ephemeris_reliability'
fname = 'V17_ephemeris_reliability.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(8, 5))
    plot_ephemeris_reliability(r, ax=ax)
    ax.set_title('V17: Ephemeris reliability')
    fig.tight_layout()
    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/19_v17_ephemeris_reliability/V17_ephemeris_reliability.png" width="820" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/19_v17_ephemeris_reliability/V17_ephemeris_reliability.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/19_v17_ephemeris_reliability/V17_ephemeris_reliability.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** phase-shift null p-value ≈ 0.00498 (z ≈ 6.13), and the best nearby-period score is at the expected period.
- **Why this is useful:** a low phase-shift p-value indicates the detection is not easily replicated by shifting the transit window to random phases.
- **Interpretation:** this supports the signal being phase-coherent, but the peak-to-next ratio (~1.21) is modest, so corroboration with robustness checks still matters.
- **Next step:** V18 sensitivity sweep across detrending/outlier strategies.

</details>
